In [1]:
import os
os.chdir('..')

%pwd

'c:\\Users\\larakim\\Documents\\Data_train\\Satellite_image_classif\\ML-pipeline-Satellite-image-classification'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
  root_dir: Path
  base_model_path: Path
  params_img_size: tuple
  params_lr: float
  params_classes: int

In [3]:
from img_classif.constants import *
from img_classif.utils.common import create_directories, read_yaml



class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH,param_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(param_path)

    def get_prepareBaseModel(self)->PrepareBaseModelConfig:
        base_model_config = PrepareBaseModelConfig(
        root_dir = self.config.base_model_prep.root_dir,
        base_model_path = self.config.base_model_prep.base_model_path,
        params_img_size = self.params.img_size,
        params_lr = self.params.lr,
        params_classes = self.params.classes)

        return base_model_config
    

In [7]:
import keras
from img_classif import logger

class PrepareBaseModel:

    def __init__(self,config:PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = keras.applications.Xception(
            weights="imagenet",  # Load weights pre-trained on ImageNet.
            input_shape=eval(self.config.params_img_size),
            include_top=False,
        )

    def update_base_model(self):
        self.model.trainable = False
        # Create new model on top
        inputs = keras.Input(shape=eval(self.config.params_img_size))

        # Pre-trained Xception weights requires that input be scaled
        # from (0, 255) to a range of (-1., +1.), the rescaling layer
        # outputs: `(inputs * scale) + offset`
        scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
        x = scale_layer(inputs)

        x = self.model(x,training=False)  
        x = keras.layers.GlobalAveragePooling2D()(x)
        x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
        predictions = keras.layers.Dense(self.config.params_classes,activation='softmax')(x)

        # Create the model
        full_model = keras.Model(inputs,predictions)

        # Print the model summary
        full_model.summary()


        full_model.save(self.config.base_model_path)
        logger.info(f'Base model successfully saved at {self.config.base_model_path}')


In [8]:
config = ConfigurationManager()
base_model_config = config.get_prepareBaseModel()
base_model_c = PrepareBaseModel(base_model_config)
base_model_c.get_base_model()
base_model_c.update_base_model()



['2024-04-08 11:43:40,781':'INFO':'common':'config\config.yaml loaded successfully']
['2024-04-08 11:43:40,786':'INFO':'common':'params.yaml loaded successfully']
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 rescaling_1 (Rescaling)     (None, 150, 150, 3)       0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                            

In [6]:
tp = (150,150,3)
tp[:2]

(150, 150)